In [3]:
# Celdas para leer directamente un archivo .arff en Python

# 1. Importar librerías necesarias
import pandas as pd
from scipy.io import arff
# Removed incorrect import of load_arff
from pathlib import Path

# 2. Definir la ruta al archivo .arff
darff_path = Path('Sapfile1.arff')  # <-- Ajusta si está en otra carpeta

# 3. Leer el .arff usando scipy
#    Devuelve una tupla (data, meta)
data_scipy, meta_scipy = arff.loadarff(str(darff_path))
# Convertir a DataFrame
df_scipy = pd.DataFrame(data_scipy)
# Decodificar columnas byte a texto si es necesario
for col in df_scipy.select_dtypes([object]):
    df_scipy[col] = df_scipy[col].str.decode('utf-8')
print("Primeras filas con scipy:")
print(df_scipy.head())

# Removed scikit-learn related code as load_arff is not available

# Opcional: mostrar información general
"""def resumen(df):
    print(df.info())
    print(df.describe())"""

# resumen(df_scipy)

# La diferencia principal entre `df_scipy` y `data_scipy` es su tipo de datos y estructura:
# - `data_scipy` es un numpy.ndarray que contiene los datos leídos directamente del archivo .arff.
# - `df_scipy` es un pandas.DataFrame que se creó a partir de `data_scipy` para facilitar el análisis y manipulación de datos.

# Ejemplo para verificar:
print(type(data_scipy))  # numpy.ndarray
print(type(df_scipy))    # pandas.DataFrame


Primeras filas con scipy:
  ge   cst   tnp   twp iap   esp arr         ms ls    as  ...  fq  mq  \
0  F     G  Good  Good  Vg  Good   Y  Unmarried  V  Paid  ...  Um  10   
1  M   OBC    Vg    Vg  Vg    Vg   N  Unmarried  V  Paid  ...  Um  Il   
2  F   OBC  Good  Good  Vg  Good   N  Unmarried  V  Paid  ...  12  10   
3  M  MOBC  Pass  Good  Vg  Good   N  Unmarried  V  Paid  ...  12  Um   
4  M     G  Good  Good  Vg    Vg   N  Unmarried  V  Paid  ...  10  12   

         fo         mo       nf       sh       ss   me       tt      atd  
0    Farmer  Housewife    Large     Poor     Govt  Asm    Small     Good  
1   Service    Service    Small     Poor     Govt  Asm  Average  Average  
2   Service  Housewife  Average  Average     Govt  Asm    Large     Good  
3  Business   Business    Large     Poor     Govt  Asm  Average  Average  
4   Service  Housewife    Large     Poor  Private  Asm    Small     Good  

[5 rows x 22 columns]
<class 'numpy.ndarray'>
<class 'pandas.core.frame.DataFrame'>


Punto 4: Transformaciones y escalado

In [12]:
# Punto 4: Transformaciones y escalado de variables
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# 1) Coerción: intenta convertir cada columna a float, los errores serán NaN
df_coerced = df_scipy.apply(pd.to_numeric, errors='coerce')

# 2) Detecta las columnas que tienen ALGÚN dato numérico
numeric_cols = [c for c in df_coerced.columns if df_coerced[c].notna().any()]
print("Columnas tras coerción con algún valor numérico:", numeric_cols)

# 3) Imputa NaNs con la mediana de cada columna
df_num = df_coerced[numeric_cols].fillna(df_coerced[numeric_cols].median())

# 4) Escalado
scaler_std = StandardScaler()
X_std = scaler_std.fit_transform(df_num)
print("Escalado estándar OK, shape:", X_std.shape)

scaler_mm = MinMaxScaler()
X_mm = scaler_mm.fit_transform(df_num)
print("Escalado Min-Max OK, shape:", X_mm.shape)



Columnas tras coerción con algún valor numérico: ['fq', 'mq']
Escalado estándar OK, shape: (131, 2)
Escalado Min-Max OK, shape: (131, 2)


**asd**


In [13]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

# --- 1) Reconstruir DataFrame con escalado estándar ---
df_std = pd.DataFrame(X_std, columns=numeric_cols)

# --- 2) Añadir variables categóricas tal cual estaban (por ejemplo 'class') ---
cat_cols = df_scipy.select_dtypes(include=['object']).columns.tolist()
df_model = pd.concat([df_std, df_scipy[cat_cols].reset_index(drop=True)], axis=1)

# --- 3) One-hot encoding de las categóricas ---
df_model = pd.get_dummies(df_model, drop_first=True)

# --- 4) Separar X e y ---
X = df_model.drop('class_Appropriate' if 'class_Appropriate' in df_model.columns else 'class', axis=1)
y = pd.get_dummies(df_scipy['class'], drop_first=True)  # ajusta según nombres de tu target

# --- 5) Importancias con Random Forest ---
rf = RandomForestClassifier(random_state=42)
rf.fit(X, y.values.ravel())  # si y es DataFrame, conviértelo a 1D
importances = pd.Series(rf.feature_importances_, index=X.columns)
importances = importances.sort_values(ascending=False)

print("Top 10 features más importantes:")
print(importances.head(10))


KeyError: "['class'] not found in axis"

In [14]:
# 1) Muestra las columnas de df_model
print("Columnas en df_model:", df_model.columns.tolist())

# 2) Intenta separar X e y y captura el error
try:
    # Ajusta aquí 'class' por el nombre de tu target si difiere
    X = df_model.drop('class_Appropriate' if 'class_Appropriate' in df_model.columns else 'class', axis=1)
    y = pd.get_dummies(df_scipy['class'], drop_first=True) 
    print("Separación OK")
except Exception as e:
    import traceback; traceback.print_exc()


Columnas en df_model: ['fq', 'mq', 'ge_M', 'cst_MOBC', 'cst_OBC', 'cst_SC', 'cst_ST', 'tnp_Good', 'tnp_Pass', 'tnp_Vg', 'twp_Good', 'twp_Pass', 'twp_Vg', 'iap_Good', 'iap_Pass', 'iap_Vg', 'esp_Good', 'esp_Pass', 'esp_Vg', 'arr_Y', 'ls_V', 'as_Paid', 'fmi_High', 'fmi_Low', 'fmi_Medium', 'fmi_Vh', 'fs_Large', 'fs_Small', 'fq_12', 'fq_Degree', 'fq_Il', 'fq_Pg', 'fq_Um', 'mq_12', 'mq_Degree', 'mq_Il', 'mq_Pg', 'mq_Um', 'fo_Farmer', 'fo_Others', 'fo_Retired', 'fo_Service', 'mo_Housewife', 'mo_Others', 'mo_Retired', 'mo_Service', 'nf_Large', 'nf_Small', 'sh_Good', 'sh_Poor', 'ss_Private', 'me_Ben', 'me_Eng', 'me_Hin', 'tt_Large', 'tt_Small', 'atd_Good', 'atd_Poor']


Traceback (most recent call last):
  File "C:\Users\tester\AppData\Local\Temp\ipykernel_17236\2578226416.py", line 7, in <module>
    X = df_model.drop('class_Appropriate' if 'class_Appropriate' in df_model.columns else 'class', axis=1)
  File "c:\Users\tester\AppData\Local\Programs\Python\Python313\Lib\site-packages\pandas\core\frame.py", line 5581, in drop
    return super().drop(
           ~~~~~~~~~~~~^
        labels=labels,
        ^^^^^^^^^^^^^^
    ...<5 lines>...
        errors=errors,
        ^^^^^^^^^^^^^^
    )
    ^
  File "c:\Users\tester\AppData\Local\Programs\Python\Python313\Lib\site-packages\pandas\core\generic.py", line 4788, in drop
    obj = obj._drop_axis(labels, axis, level=level, errors=errors)
  File "c:\Users\tester\AppData\Local\Programs\Python\Python313\Lib\site-packages\pandas\core\generic.py", line 4830, in _drop_axis
    new_axis = axis.drop(labels, errors=errors)
  File "c:\Users\tester\AppData\Local\Programs\Python\Python313\Lib\site-packages\pandas\cor